In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise import AlgoBase # <--- 이 부분이 반드시 임포트되어 있어야 합니다!
from surprise import accuracy
from surprise.model_selection import GridSearchCV
import random
import time

# --- 1. 데이터 로드 및 전처리 (감성 벡터 추출 포함) ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 'sentiment_vector' 컬럼이 리스트 형태인지 확인하고, 아니면 15개의 0으로 채웁니다.
# 이는 결측치나 잘못된 형식의 감성 벡터를 처리하는 데 유용합니다.
data['sentiment_vector'] = data['sentiment_vector'].apply(lambda x: x if isinstance(x, list) and len(x) == 15 else [0.0]*15)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# --- 평점 데이터 집계 ---
# 동일한 사용자-비즈니스 쌍에 여러 리뷰가 있다면 평균 평점을 계산합니다.
data_aggregated_ratings = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

# --- 비즈니스별 평균 감성 벡터 집계 ---
# 각 'business_encoded'에 해당하는 모든 리뷰의 'sentiment_vector'를 평균합니다.
# 이 결과는 딕셔너리 형태로 저장되어 나중에 해당 비즈니스의 감성 특징으로 사용됩니다.
# {business_encoded_id: [평균_감성_벡터]}
business_sentiment_vectors_map = data_clean.groupby('business_encoded')['sentiment_vector'].apply(lambda x: np.mean(list(x), axis=0)).to_dict()

print(f"총 고유 사용자 수: {data_aggregated_ratings['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated_ratings['business_encoded'].nunique():,}개")
print(f"총 집계된 평점 데이터 수: {len(data_aggregated_ratings):,}개")
print(f"감성 벡터가 집계된 고유 비즈니스 수: {len(business_sentiment_vectors_map):,}개")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점 데이터 수: 428,953개
감성 벡터가 집계된 고유 비즈니스 수: 6,831개


In [18]:
# --- 2. 넷플릭스 대회식 데이터 분할 (훈련/테스트 세트) ---

# 집계된 평점 데이터를 훈련 (80%) 및 테스트 (20%) 세트로 나눕니다.
# random_state를 고정하여 항상 동일한 분할을 얻습니다.
train_df, test_df = train_test_split(data_aggregated_ratings, test_size=0.2, random_state=42)

print(f"\n훈련 데이터 수: {len(train_df):,}개")
print(f"테스트 데이터 수: {len(test_df):,}개")

# Surprise 라이브러리용 데이터 로드
# 평점 스케일은 1부터 5까지입니다.
reader = Reader(rating_scale=(1, 5))

# 훈련 세트 로드 (Surprise는 build_full_trainset()을 사용하여 내부 데이터 구조를 만듭니다)
trainset_surprise = Dataset.load_from_df(train_df[['user_encoded', 'business_encoded', 'stars']], reader).build_full_trainset()

# 테스트 세트 로드 (Surprise는 예측 시 (user_id, item_id, true_rating) 튜플 리스트를 선호합니다)
testset_surprise = list(test_df.apply(lambda x: (x['user_encoded'], x['business_encoded'], x['stars']), axis=1))


훈련 데이터 수: 343,162개
테스트 데이터 수: 85,791개


In [19]:
# --- 3. SVD 모델 하이퍼파라미터 튜닝 (GridSearchCV) ---

print("\n--- SVD 모델 하이퍼파라미터 튜닝 (GridSearchCV) 시작 ---")

# GridSearchCV는 Dataset 객체를 직접 받습니다 (build_full_trainset() 전 단계의 객체)
# 이를 위해 전체 데이터셋(data_aggregated_ratings)에서 Dataset 객체를 다시 만듭니다.
full_dataset_surprise = Dataset.load_from_df(data_aggregated_ratings[['user_encoded', 'business_encoded', 'stars']], reader)

# SVD 모델의 탐색할 하이퍼파라미터 그리드를 정의합니다.
param_grid_svd = {
    'n_factors': [1], # n_factors=1이 최적이었으므로 1만 남겨둡니다.
    'n_epochs': [10, 20, 30, 40, 50], # 에포크 수를 더 넓게 탐색
    'lr_all': [0.001, 0.002, 0.005, 0.01], # 학습률 조정
    'reg_all': [0.005, 0.01, 0.02, 0.05] # 정규화 강도 조정
}

# GridSearchCV를 초기화합니다.
# SVD 모델을 사용하고, RMSE와 MAE를 측정하며, 3-폴드 교차 검증을 수행하고, 모든 CPU 코어를 사용합니다.
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# GridSearchCV를 전체 데이터셋에 대해 실행하여 최적의 파라미터를 찾습니다.
gs_svd.fit(full_dataset_surprise)

# 최적의 RMSE 및 MAE 점수와 해당 파라미터를 출력합니다.
print(f"\n--- SVD GridSearchCV 결과 ---")
print(f"최적 RMSE 점수: {gs_svd.best_score['rmse']:.4f}")
print(f"최적 RMSE 파라미터: {gs_svd.best_params['rmse']}")
print(f"최적 MAE 점수: {gs_svd.best_score['mae']:.4f}")
print(f"최적 MAE 파라미터: {gs_svd.best_params['mae']}")

# 최적 SVD 파라미터 저장
best_svd_params = gs_svd.best_params['rmse'] # RMSE 기준 최적 파라미터 사용


--- SVD 모델 하이퍼파라미터 튜닝 (GridSearchCV) 시작 ---

--- SVD GridSearchCV 결과 ---
최적 RMSE 점수: 1.0270
최적 RMSE 파라미터: {'n_factors': 1, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.05}
최적 MAE 점수: 0.7974
최적 MAE 파라미터: {'n_factors': 1, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.05}


In [23]:
# --- 4. 커스텀 SVD (ABSA 통합) 모델 정의 및 평가 ---

class MySVDWithABSA(AlgoBase):
    """
    ABSA 감성 벡터를 SVD의 아이템 잠재 요인 학습에 통합하는 커스텀 SVD 알고리즘.
    아이템 잠재 요인 qi가 해당 아이템의 감성 벡터 si와 유사하도록 정규화 항을 추가합니다.
    """
    def __init__(self, n_factors=1, n_epochs=20, lr_all=0.005, reg_all=0.02,
                 reg_s=0.01, random_state=None):
        """
        Args:
            n_factors (int): 잠재 요인의 수.
            n_epochs (int): SGD 에포크 수.
            lr_all (float): 학습률.
            reg_all (float): 모든 매개변수에 대한 기본 정규화 강도.
            reg_s (float): 감성 벡터 정규화에 대한 강도 (추가된 파라미터).
            random_state (int): 재현성을 위한 시드.
        """
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr_all = lr_all
        self.reg_all = reg_all
        self.reg_s = reg_s # ABSA 통합을 위한 새로운 정규화 계수
        self.random_state = random_state

        if self.random_state is not None:
            random.seed(self.random_state)
            np.random.seed(self.random_state)

        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.bu = np.zeros(trainset.n_users, np.double)
        self.bi = np.zeros(trainset.n_items, np.double)
        self.pu = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        self.qi = np.random.normal(0, .1, (trainset.n_items, self.n_factors))

        self.inner_item_sentiment_vectors = {}
        for business_encoded_id, sentiment_vec in business_sentiment_vectors_map.items():
            try:
                inner_iid = trainset.to_inner_iid(business_encoded_id)
                self.inner_item_sentiment_vectors[inner_iid] = np.array(sentiment_vec, dtype=np.double)
            except ValueError:
                pass

        self.sentiment_dim = len(next(iter(business_sentiment_vectors_map.values()))) if business_sentiment_vectors_map else 0

        # SGD (Stochastic Gradient Descent) 학습 루프
        for current_epoch in range(self.n_epochs):
            print(f"Epoch {current_epoch + 1}/{self.n_epochs}...")
            
            # --- 수정된 부분: trainset.all_ratings()의 제너레이터 출력을 리스트로 변환 ---
            current_ratings = list(trainset.all_ratings()) # 제너레이터 출력을 리스트로 변환
            random.shuffle(current_ratings) # 리스트를 셔플
            # ----------------------------------------------------------------------------------

            for u, i, r in current_ratings: # 셔플된 리스트를 반복
                dot_product = np.dot(self.qi[i], self.pu[u])
                pred_r = self.trainset.global_mean + self.bu[u] + self.bi[i] + dot_product

                err = r - pred_r

                self.bu[u] += self.lr_all * (err - self.reg_all * self.bu[u])
                self.bi[i] += self.lr_all * (err - self.reg_all * self.bi[i])

                pu_old = self.pu[u, :]
                self.pu[u, :] += self.lr_all * (err * self.qi[i] - self.reg_all * self.pu[u, :])

                qi_old = self.qi[i, :]
                
                # SVD 기본 업데이트
                self.qi[i, :] += self.lr_all * (err * pu_old - self.reg_all * self.qi[i, :])

                # !!! ABSA 감성 벡터 정규화 항 추가 (핵심 변경점) !!!
                # 해당 아이템의 감성 벡터가 있다면, qi를 감성 벡터 방향으로 당깁니다.
                if i in self.inner_item_sentiment_vectors:
                    s_i = self.inner_item_sentiment_vectors[i]
                    
                    # n_factors가 sentiment_dim (15)과 동일하다고 가정하고 직접 정규화
                    if self.n_factors == self.sentiment_dim:
                         self.qi[i, :] += self.lr_all * self.reg_s * (s_i - self.qi[i, :])
                    else:
                        # n_factors != sentiment_dim 일 경우, 이 항은 직접적인 빼기 연산이 어렵습니다.
                        # 이 부분은 나중에 더 정교하게 만들 수 있습니다.
                        pass
                
        return self

    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            return self.trainset.global_mean

        return self.trainset.global_mean + self.bu[u] + self.bi[i] + np.dot(self.qi[i], self.pu[u])

# --- 커스텀 SVD (ABSA 통합) 모델 학습 및 평가 ---
print("\n--- 커스텀 SVD (ABSA 통합) 모델 학습 및 평가 시작 ---")

# n_factors는 sentiment_vector의 차원인 15로 설정하는 것이 가장 적합합니다.
# 이렇게 해야 위에서 감성 벡터 정규화 항이 의미를 가집니다.
# reg_s는 감성 벡터 정규화 강도입니다. 이 값을 튜닝해야 합니다.
algo_absa_svd = MySVDWithABSA(n_factors=15, n_epochs=50, lr_all=0.005, reg_all=0.02, reg_s=0.1, random_state=42)

# 모델 학습
start_time = time.time()
algo_absa_svd.fit(trainset_surprise)
end_time = time.time()
print(f"학습 시간: {end_time - start_time:.2f}초")

# 테스트 세트에 대한 예측 생성
predictions_absa_svd = algo_absa_svd.test(testset_surprise)

# RMSE 및 MAE 계산
rmse_absa_svd = accuracy.rmse(predictions_absa_svd, verbose=False)
mae_absa_svd = accuracy.mae(predictions_absa_svd, verbose=False)

print(f"\n--- 커스텀 SVD (ABSA 통합) 결과 ---")
print(f"RMSE: {rmse_absa_svd:.4f}")
print(f"MAE: {mae_absa_svd:.4f}")

# SVD 단독 모델 (최적) 결과와 비교
print("\n--- SVD 단독 모델 (최적) 결과와 비교 ---")
print(f"SVD 단독 (최적) RMSE: {gs_svd.best_score['rmse']:.4f}")
print(f"SVD 단독 (최적) MAE: {gs_svd.best_score['mae']:.4f}")


--- 커스텀 SVD (ABSA 통합) 모델 학습 및 평가 시작 ---
Epoch 1/50...
Epoch 2/50...
Epoch 3/50...
Epoch 4/50...
Epoch 5/50...
Epoch 6/50...
Epoch 7/50...
Epoch 8/50...
Epoch 9/50...
Epoch 10/50...
Epoch 11/50...
Epoch 12/50...
Epoch 13/50...
Epoch 14/50...
Epoch 15/50...
Epoch 16/50...
Epoch 17/50...
Epoch 18/50...
Epoch 19/50...
Epoch 20/50...
Epoch 21/50...
Epoch 22/50...
Epoch 23/50...
Epoch 24/50...
Epoch 25/50...
Epoch 26/50...
Epoch 27/50...
Epoch 28/50...
Epoch 29/50...
Epoch 30/50...
Epoch 31/50...
Epoch 32/50...
Epoch 33/50...
Epoch 34/50...
Epoch 35/50...
Epoch 36/50...
Epoch 37/50...
Epoch 38/50...
Epoch 39/50...
Epoch 40/50...
Epoch 41/50...
Epoch 42/50...
Epoch 43/50...
Epoch 44/50...
Epoch 45/50...
Epoch 46/50...
Epoch 47/50...
Epoch 48/50...
Epoch 49/50...
Epoch 50/50...
학습 시간: 174.98초

--- 커스텀 SVD (ABSA 통합) 결과 ---
RMSE: 1.0395
MAE: 0.7959

--- SVD 단독 모델 (최적) 결과와 비교 ---
SVD 단독 (최적) RMSE: 1.0270
SVD 단독 (최적) MAE: 0.7974


In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise import AlgoBase # 커스텀 알고리즘을 위해 AlgoBase 임포트
from surprise import accuracy # RMSE/MAE 평가
from surprise.model_selection import GridSearchCV # SVD GridSearchCV에 사용
import random # 난수 생성을 위해
import time # 학습 시간 측정

# --- 1. 데이터 로드 및 전처리 (감성 벡터 추출 포함) ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 'sentiment_vector' 컬럼이 리스트 형태인지 확인하고, 아니면 15개의 0으로 채웁니다.
# 이는 결측치나 잘못된 형식의 감성 벡터를 처리하는 데 유용합니다.
data['sentiment_vector'] = data['sentiment_vector'].apply(lambda x: x if isinstance(x, list) and len(x) == 15 else [0.0]*15)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# --- 평점 데이터 집계 ---
# 동일한 사용자-비즈니스 쌍에 여러 리뷰가 있다면 평균 평점을 계산합니다.
data_aggregated_ratings = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

# --- 비즈니스별 평균 감성 벡터 집계 ---
# 각 'business_encoded'에 해당하는 모든 리뷰의 'sentiment_vector'를 평균합니다.
# 이 결과는 딕셔너리 형태로 저장되어 나중에 해당 비즈니스의 감성 특징으로 사용됩니다.
# {business_encoded_id: [평균_감성_벡터]}
business_sentiment_vectors_map = data_clean.groupby('business_encoded')['sentiment_vector'].apply(lambda x: np.mean(list(x), axis=0)).to_dict()

print(f"총 고유 사용자 수: {data_aggregated_ratings['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated_ratings['business_encoded'].nunique():,}개")
print(f"총 집계된 평점 데이터 수: {len(data_aggregated_ratings):,}개")
print(f"감성 벡터가 집계된 고유 비즈니스 수: {len(business_sentiment_vectors_map):,}개")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점 데이터 수: 428,953개
감성 벡터가 집계된 고유 비즈니스 수: 6,831개


In [26]:
# --- 2. 넷플릭스 대회식 데이터 분할 (훈련/테스트 세트) ---

# 집계된 평점 데이터를 훈련 (80%) 및 테스트 (20%) 세트로 나눕니다.
# random_state를 고정하여 항상 동일한 분할을 얻습니다.
train_df, test_df = train_test_split(data_aggregated_ratings, test_size=0.2, random_state=42)

print(f"\n훈련 데이터 수: {len(train_df):,}개")
print(f"테스트 데이터 수: {len(test_df):,}개")

# Surprise 라이브러리용 데이터 로드
# 평점 스케일은 1부터 5까지입니다.
reader = Reader(rating_scale=(1, 5))

# 훈련 세트 로드 (Surprise는 build_full_trainset()을 사용하여 내부 데이터 구조를 만듭니다)
trainset_surprise = Dataset.load_from_df(train_df[['user_encoded', 'business_encoded', 'stars']], reader).build_full_trainset()

# 테스트 세트 로드 (Surprise는 예측 시 (user_id, item_id, true_rating) 튜플 리스트를 선호합니다)
testset_surprise = list(test_df.apply(lambda x: (x['user_encoded'], x['business_encoded'], x['stars']), axis=1))


훈련 데이터 수: 343,162개
테스트 데이터 수: 85,791개


In [28]:
# --- 3. SVD 모델 하이퍼파라미터 튜닝 (GridSearchCV) ---

print("\n--- SVD 모델 하이퍼파라미터 튜닝 (GridSearchCV) 시작 ---")

# GridSearchCV는 Dataset 객체를 직접 받습니다 (build_full_trainset() 전 단계의 객체)
# 이를 위해 전체 데이터셋(data_aggregated_ratings)에서 Dataset 객체를 다시 만듭니다.
full_dataset_surprise = Dataset.load_from_df(data_aggregated_ratings[['user_encoded', 'business_encoded', 'stars']], reader)

# SVD 모델의 탐색할 하이퍼파라미터 그리드를 정의합니다.
param_grid_svd = {
    'n_factors': [1], # n_factors=1이 최적이었으므로 1만 남겨둡니다.
    'n_epochs': [10, 20, 30, 40, 50], # 에포크 수를 더 넓게 탐색
    'lr_all': [0.001, 0.002, 0.005, 0.01], # 학습률 조정
    'reg_all': [0.005, 0.01, 0.02, 0.05] # 정규화 강도 조정
}

# GridSearchCV를 초기화합니다.
# SVD 모델을 사용하고, RMSE와 MAE를 측정하며, 3-폴드 교차 검증을 수행하고, 모든 CPU 코어를 사용합니다.
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# GridSearchCV를 전체 데이터셋에 대해 실행하여 최적의 파라미터를 찾습니다.
gs_svd.fit(full_dataset_surprise)

# 최적의 RMSE 및 MAE 점수와 해당 파라미터를 출력합니다.
print(f"\n--- SVD GridSearchCV 결과 ---")
print(f"최적 RMSE 점수: {gs_svd.best_score['rmse']:.4f}")
print(f"최적 RMSE 파라미터: {gs_svd.best_params['rmse']}")
print(f"최적 MAE 점수: {gs_svd.best_score['mae']:.4f}")
print(f"최적 MAE 파라미터: {gs_svd.best_params['mae']}")

# 최적 SVD 파라미터 저장
best_svd_params = gs_svd.best_params['rmse'] # RMSE 기준 최적 파라미터 사용


--- SVD 모델 하이퍼파라미터 튜닝 (GridSearchCV) 시작 ---

--- SVD GridSearchCV 결과 ---
최적 RMSE 점수: 1.0269
최적 RMSE 파라미터: {'n_factors': 1, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.05}
최적 MAE 점수: 0.7976
최적 MAE 파라미터: {'n_factors': 1, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.05}


In [30]:
# --- 4. 커스텀 SVD (ABSA 통합) 모델 정의 및 하이퍼파라미터 튜닝 ---

class MySVDWithABSA(AlgoBase):
    """
    ABSA 감성 벡터를 SVD의 아이템 잠재 요인 학습에 통합하는 커스텀 SVD 알고리즘.
    아이템 잠재 요인 qi가 해당 아이템의 감성 벡터 si와 유사하도록 정규화 항을 추가합니다.
    """
    def __init__(self, n_factors=1, n_epochs=20, lr_all=0.005, reg_all=0.02,
                 reg_s=0.01, random_state=None):
        """
        Args:
            n_factors (int): 잠재 요인의 수.
            n_epochs (int): SGD 에포크 수.
            lr_all (float): 학습률.
            reg_all (float): 모든 매개변수에 대한 기본 정규화 강도.
            reg_s (float): 감성 벡터 정규화에 대한 강도 (추가된 파라미터).
            random_state (int): 재현성을 위한 시드.
        """
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr_all = lr_all
        self.reg_all = reg_all
        self.reg_s = reg_s # ABSA 통합을 위한 새로운 정규화 계수
        self.random_state = random_state

        if self.random_state is not None:
            random.seed(self.random_state)
            np.random.seed(self.random_state)

        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.bu = np.zeros(trainset.n_users, np.double)
        self.bi = np.zeros(trainset.n_items, np.double)
        self.pu = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        self.qi = np.random.normal(0, .1, (trainset.n_items, self.n_factors))

        self.inner_item_sentiment_vectors = {}
        for business_encoded_id, sentiment_vec in business_sentiment_vectors_map.items():
            try:
                inner_iid = trainset.to_inner_iid(business_encoded_id)
                self.inner_item_sentiment_vectors[inner_iid] = np.array(sentiment_vec, dtype=np.double)
            except ValueError:
                pass

        self.sentiment_dim = len(next(iter(business_sentiment_vectors_map.values()))) if business_sentiment_vectors_map else 0

        # SGD (Stochastic Gradient Descent) 학습 루프
        for current_epoch in range(self.n_epochs):
            # print(f"Epoch {current_epoch + 1}/{self.n_epochs}...") # 튜닝 중에는 주석 처리하여 출력 줄임
            
            current_ratings = list(trainset.all_ratings()) # 제너레이터 출력을 리스트로 변환
            random.shuffle(current_ratings) # 리스트를 셔플

            for u, i, r in current_ratings: # 셔플된 리스트를 반복
                dot_product = np.dot(self.qi[i], self.pu[u])
                pred_r = self.trainset.global_mean + self.bu[u] + self.bi[i] + dot_product

                err = r - pred_r

                self.bu[u] += self.lr_all * (err - self.reg_all * self.bu[u])
                self.bi[i] += self.lr_all * (err - self.reg_all * self.bi[i])

                pu_old = self.pu[u, :]
                self.pu[u, :] += self.lr_all * (err * self.qi[i] - self.reg_all * self.pu[u, :])

                qi_old = self.qi[i, :]
                
                # SVD 기본 업데이트
                self.qi[i, :] += self.lr_all * (err * pu_old - self.reg_all * self.qi[i, :])

                # !!! ABSA 감성 벡터 정규화 항 추가 (핵심 변경점) !!!
                if i in self.inner_item_sentiment_vectors:
                    s_i = self.inner_item_sentiment_vectors[i]
                    
                    # n_factors가 sentiment_dim (15)과 동일하다고 가정하고 직접 정규화
                    if self.n_factors == self.sentiment_dim:
                         # 감성 벡터를 잠재 요인 공간으로 매핑하는 W 행렬을 직접 학습하지 않고,
                         # 단순히 qi가 si와 가까워지도록 유도합니다.
                         # 이 방식은 n_factors와 sentiment_dim이 동일할 때 가장 효과적입니다.
                         self.qi[i, :] += self.lr_all * self.reg_s * (s_i - self.qi[i, :])
                    else:
                        # n_factors != sentiment_dim 인 경우
                        # 이 부분은 지금은 비활성화합니다.
                        # 만약 n_factors != sentiment_dim 이라면 s_i를 n_factors 차원으로 투영하는 W 행렬이 필요합니다.
                        # 예를 들어, self.qi[i, :] += self.lr_all * self.reg_s * (np.dot(self.W_map, s_i) - self.qi[i, :])
                        # 이 경우 self.W_map도 SGD로 함께 학습해야 합니다.
                        pass
                
        return self

    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            return self.trainset.global_mean

        return self.trainset.global_mean + self.bu[u] + self.bi[i] + np.dot(self.qi[i], self.pu[u])


print("\n--- 커스텀 SVD (ABSA 통합) 모델 하이퍼파라미터 튜닝 (GridSearchCV) 시작 ---")

# 튜닝할 MySVDWithABSA 모델의 하이퍼파라미터 그리드를 정의합니다.
param_grid_absa_svd = {
    'n_factors': [10, 15, 20],  # 잠재 요인 수
    'n_epochs': [30, 50, 70],    # 에포크 수
    'lr_all': [0.002, 0.005],  # 학습률
    'reg_all': [0.01, 0.02],   # 일반 정규화 강도
    'reg_s': [0.001, 0.01, 0.1, 0.5], # ABSA 감성 벡터 정규화 강도 (중요)
    'random_state': [42] # 재현성을 위해 고정
}

# GridSearchCV를 초기화합니다.
# MySVDWithABSA 모델을 사용하고, RMSE와 MAE를 측정하며, 3-폴드 교차 검증을 수행하고, 모든 CPU 코어를 사용합니다.
# GridSearchCV는 학습 데이터 (full_dataset_surprise)에서 각 폴드를 나누어 검증을 수행합니다.
# verbose 인자를 제거했습니다.
gs_absa_svd = GridSearchCV(MySVDWithABSA, param_grid_absa_svd, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# GridSearchCV를 실행하여 최적의 파라미터를 찾습니다.
start_time = time.time()
gs_absa_svd.fit(full_dataset_surprise) # 전체 데이터셋을 사용하여 Cross-Validation 수행
end_time = time.time()
print(f"GridSearchCV 튜닝 시간: {end_time - start_time:.2f}초")

# 최적의 RMSE 및 MAE 점수와 해당 파라미터를 출력합니다.
print(f"\n--- 커스텀 SVD (ABSA 통합) GridSearchCV 결과 ---")
print(f"최적 RMSE 점수: {gs_absa_svd.best_score['rmse']:.4f}")
print(f"최적 RMSE 파라미터: {gs_absa_svd.best_params['rmse']}")
print(f"최적 MAE 점수: {gs_absa_svd.best_score['mae']:.4f}")
print(f"최적 MAE 파라미터: {gs_absa_svd.best_params['mae']}")

# 최적 ABSA 통합 SVD 파라미터 저장
best_absa_svd_params = gs_absa_svd.best_params['rmse'] # RMSE 기준 최적 파라미터 사용

# --- 최적 파라미터로 커스텀 SVD (ABSA 통합) 모델 최종 학습 및 평가 ---
print(f"\n--- 최적 파라미터로 커스텀 SVD (ABSA 통합) 모델 최종 학습 중 (훈련 세트 사용) ---")

final_absa_svd_algo = MySVDWithABSA(**best_absa_svd_params)

start_time = time.time()
final_absa_svd_algo.fit(trainset_surprise) # 2번 셀에서 생성된 훈련 데이터 (trainset_surprise)로 학습
end_time = time.time()
print(f"최종 모델 학습 시간: {end_time - start_time:.2f}초")


# 테스트 세트에 대한 예측 생성
predictions_final_absa_svd = final_absa_svd_algo.test(testset_surprise)

# RMSE 및 MAE 계산
rmse_final_absa_svd = accuracy.rmse(predictions_final_absa_svd, verbose=False)
mae_final_absa_svd = accuracy.mae(predictions_final_absa_svd, verbose=False)

print(f"\n--- 커스텀 SVD (ABSA 통합, 최적 파라미터) 최종 결과 ---")
print(f"RMSE: {rmse_final_absa_svd:.4f}")
print(f"MAE: {mae_final_absa_svd:.4f}")

# SVD 단독 모델 (최적) 결과와 비교
print("\n--- SVD 단독 모델 (최적) 결과와 비교 ---")
print(f"SVD 단독 (최적) RMSE: {gs_svd.best_score['rmse']:.4f}")
print(f"SVD 단독 (최적) MAE: {gs_svd.best_score['mae']:.4f}")


--- 커스텀 SVD (ABSA 통합) 모델 하이퍼파라미터 튜닝 (GridSearchCV) 시작 ---
GridSearchCV 튜닝 시간: 6263.37초

--- 커스텀 SVD (ABSA 통합) GridSearchCV 결과 ---
최적 RMSE 점수: 1.0301
최적 RMSE 파라미터: {'n_factors': 10, 'n_epochs': 50, 'lr_all': 0.002, 'reg_all': 0.02, 'reg_s': 0.01, 'random_state': 42}
최적 MAE 점수: 0.7977
최적 MAE 파라미터: {'n_factors': 15, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.01, 'reg_s': 0.5, 'random_state': 42}

--- 최적 파라미터로 커스텀 SVD (ABSA 통합) 모델 최종 학습 중 (훈련 세트 사용) ---
최종 모델 학습 시간: 137.90초

--- 커스텀 SVD (ABSA 통합, 최적 파라미터) 최종 결과 ---
RMSE: 1.0241
MAE: 0.7974

--- SVD 단독 모델 (최적) 결과와 비교 ---
SVD 단독 (최적) RMSE: 1.0269
SVD 단독 (최적) MAE: 0.7976


In [31]:
# 이전에 정의된 calculate_metrics 함수가 이 셀 실행 전에 정의되어 있어야 합니다.
# 또한, predictions_final_absa_svd와 testset_surprise 변수가 이미 생성되어 있어야 합니다.

print("\n--- 커스텀 SVD (ABSA 통합, 최적 파라미터) 순위 기반 지표 ---")

# Precision@10, Recall@10, NDCG@10 계산
precision_absa_svd, recall_absa_svd, ndcg_absa_svd = calculate_metrics(predictions_final_absa_svd, testset_surprise, k=10)

print(f"ABSA 통합 SVD Precision@10: {precision_absa_svd:.4f}")
print(f"ABSA 통합 SVD Recall@10: {recall_absa_svd:.4f}")
print(f"ABSA 통합 SVD NDCG@10: {ndcg_absa_svd:.4f}")


--- 커스텀 SVD (ABSA 통합, 최적 파라미터) 순위 기반 지표 ---


NameError: name 'calculate_metrics' is not defined

In [ ]:
# --- 4. 커스텀 SVD (ABSA 통합) 모델 정의 및 하이퍼파라미터 튜닝 ---

class MySVDWithABSA(AlgoBase):
    """
    ABSA 감성 벡터를 SVD의 아이템 잠재 요인 학습에 통합하는 커스텀 SVD 알고리즘.
    아이템 잠재 요인 qi가 해당 아이템의 감성 벡터 si와 유사하도록 정규화 항을 추가합니다.
    """
    def __init__(self, n_factors=1, n_epochs=20, lr_all=0.005, reg_all=0.02,
                 reg_s=0.01, random_state=None):
        """
        Args:
            n_factors (int): 잠재 요인의 수.
            n_epochs (int): SGD 에포크 수.
            lr_all (float): 학습률.
            reg_all (float): 모든 매개변수에 대한 기본 정규화 강도.
            reg_s (float): 감성 벡터 정규화에 대한 강도 (추가된 파라미터).
            random_state (int): 재현성을 위한 시드.
        """
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.lr_all = lr_all
        self.reg_all = reg_all
        self.reg_s = reg_s # ABSA 통합을 위한 새로운 정규화 계수
        self.random_state = random_state

        if self.random_state is not None:
            random.seed(self.random_state)
            np.random.seed(self.random_state)

        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.bu = np.zeros(trainset.n_users, np.double)
        self.bi = np.zeros(trainset.n_items, np.double)
        self.pu = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        self.qi = np.random.normal(0, .1, (trainset.n_items, self.n_factors))

        self.inner_item_sentiment_vectors = {}
        for business_encoded_id, sentiment_vec in business_sentiment_vectors_map.items():
            try:
                inner_iid = trainset.to_inner_iid(business_encoded_id)
                self.inner_item_sentiment_vectors[inner_iid] = np.array(sentiment_vec, dtype=np.double)
            except ValueError:
                pass

        self.sentiment_dim = len(next(iter(business_sentiment_vectors_map.values()))) if business_sentiment_vectors_map else 0

        # SGD (Stochastic Gradient Descent) 학습 루프
        for current_epoch in range(self.n_epochs):
            # print(f"Epoch {current_epoch + 1}/{self.n_epochs}...") # 튜닝 중에는 주석 처리하여 출력 줄임
            
            current_ratings = list(trainset.all_ratings()) # 제너레이터 출력을 리스트로 변환
            random.shuffle(current_ratings) # 리스트를 셔플

            for u, i, r in current_ratings: # 셔플된 리스트를 반복
                dot_product = np.dot(self.qi[i], self.pu[u])
                pred_r = self.trainset.global_mean + self.bu[u] + self.bi[i] + dot_product

                err = r - pred_r

                self.bu[u] += self.lr_all * (err - self.reg_all * self.bu[u])
                self.bi[i] += self.lr_all * (err - self.reg_all * self.bi[i])

                pu_old = self.pu[u, :]
                self.pu[u, :] += self.lr_all * (err * self.qi[i] - self.reg_all * self.pu[u, :])

                qi_old = self.qi[i, :]
                
                # SVD 기본 업데이트
                self.qi[i, :] += self.lr_all * (err * pu_old - self.reg_all * self.qi[i, :])

                # !!! ABSA 감성 벡터 정규화 항 추가 (핵심 변경점) !!!
                if i in self.inner_item_sentiment_vectors:
                    s_i = self.inner_item_sentiment_vectors[i]
                    
                    # n_factors가 sentiment_dim (15)과 동일하다고 가정하고 직접 정규화
                    if self.n_factors == self.sentiment_dim:
                         self.qi[i, :] += self.lr_all * self.reg_s * (s_i - self.qi[i, :])
                    else:
                        # n_factors != sentiment_dim 인 경우
                        pass
                
        return self

    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            return self.trainset.global_mean

        return self.trainset.global_mean + self.bu[u] + self.bi[i] + np.dot(self.qi[i], self.pu[u])


print("\n--- 커스텀 SVD (ABSA 통합) 모델 하이퍼파라미터 세밀 튜닝 (GridSearchCV) 시작 ---")

# 튜닝할 MySVDWithABSA 모델의 하이퍼파라미터 그리드를 정의합니다.
# 이전 최적 파라미터 주변으로 더 세밀하게 탐색합니다.
param_grid_absa_svd = {
    'n_factors': [8, 10, 12],  # 이전 최적: 10
    'n_epochs': [40, 50, 60],    # 이전 최적: 50
    'lr_all': [0.001, 0.002, 0.003],  # 이전 최적: 0.002
    'reg_all': [0.015, 0.02, 0.025],   # 이전 최적: 0.02
    'reg_s': [0.005, 0.01, 0.02], # 이전 최적: 0.01 (가장 중요)
    'random_state': [42] # 재현성을 위해 고정
}

# GridSearchCV를 초기화합니다.
gs_absa_svd = GridSearchCV(MySVDWithABSA, param_grid_absa_svd, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# GridSearchCV를 실행하여 최적의 파라미터를 찾습니다.
start_time = time.time()
gs_absa_svd.fit(full_dataset_surprise) # 전체 데이터셋을 사용하여 Cross-Validation 수행
end_time = time.time()
print(f"GridSearchCV 세밀 튜닝 시간: {end_time - start_time:.2f}초")

# 최적의 RMSE 및 MAE 점수와 해당 파라미터를 출력합니다.
print(f"\n--- 커스텀 SVD (ABSA 통합) 세밀 튜닝 GridSearchCV 결과 ---")
print(f"최적 RMSE 점수: {gs_absa_svd.best_score['rmse']:.4f}")
print(f"최적 RMSE 파라미터: {gs_absa_svd.best_params['rmse']}")
print(f"최적 MAE 점수: {gs_absa_svd.best_score['mae']:.4f}")
print(f"최적 MAE 파라미터: {gs_absa_svd.best_params['mae']}")

# 최적 ABSA 통합 SVD 파라미터 저장
best_absa_svd_params = gs_absa_svd.best_params['rmse'] # RMSE 기준 최적 파라미터 사용

# --- 최적 파라미터로 커스텀 SVD (ABSA 통합) 모델 최종 학습 및 평가 ---
print(f"\n--- 최적 파라미터로 커스텀 SVD (ABSA 통합) 모델 최종 학습 중 (훈련 세트 사용) ---")

final_absa_svd_algo = MySVDWithABSA(**best_absa_svd_params)

start_time = time.time()
final_absa_svd_algo.fit(trainset_surprise) # 2번 셀에서 생성된 훈련 데이터 (trainset_surprise)로 학습
end_time = time.time()
print(f"최종 모델 학습 시간: {end_time - start_time:.2f}초")


# 테스트 세트에 대한 예측 생성
predictions_final_absa_svd = final_absa_svd_algo.test(testset_surprise)

# RMSE 및 MAE 계산
rmse_final_absa_svd = accuracy.rmse(predictions_final_absa_svd, verbose=False)
mae_final_absa_svd = accuracy.mae(predictions_final_absa_svd, verbose=False)

print(f"\n--- 커스텀 SVD (ABSA 통합, 최적 파라미터) 최종 결과 ---")
print(f"RMSE: {rmse_final_absa_svd:.4f}")
print(f"MAE: {mae_final_absa_svd:.4f}")

# SVD 단독 모델 (최적) 결과와 비교
print("\n--- SVD 단독 모델 (최적) 결과와 비교 ---")
print(f"SVD 단독 (최적) RMSE: {gs_svd.best_score['rmse']:.4f}")
print(f"SVD 단독 (최적) MAE: {gs_svd.best_score['mae']:.4f}")